# Running all of the Code

In this chapter, we'll run all of the code by executing the `update.py` file, which has its contents displayed below.

## Examining `update.py`

It begins by importing all of the necessary functions and classes from `prepare.py` and `models.py`. Several parameters are defined, which will be used to instantiate `CasesModel` and `DeathsModel`. You may wish to change these values to get better results.

The `if __name__ == "__main__"` condition is a common way to "protect" code from being arbitrarily executed when a module is imported. For instance, if we have the code `import update` in another module, the code block within the `if` statement will NOT be run. 

The `if` condition isn't actually necessary here since we will not be importing `update` from other modules, but is still good to have as it signals that this file is meant for executing from the command line. The variable name `__name__` is a string that all Python modules have as attributes when they are executed or imported. If the file is executed from the command line, this string is set to `'__main__'`. Therefore, the code block within the `if` statement executes whenever this file is run from the command line.

### Command line arguments

There are two ways to execute `update.py`, which are shown below.

```bash
python update.py
python update.py 20200720
```

You can run it with or without a date which may be given in the form YYYYMMDD. Within the `if` statement's code block, the built-in `sys` module is used to retrieve the command line arguments. All the values following the word `python` that are separated by a space are considered 'arguments' including the name of the file. For instance, when you execute `python update.py 20200720` there are two arguments, `update.py` and `20200720`. Both are stored as strings in the list `sys.argv`.

The first thing we check in the `if` code block is the length of the argument list. If the length is 1, then just the name of the executable file was provided without a date. The `last_date` variable is set to `None`, which the models will interpret as the last available date when the John Hopkins repository was updated. This is the most common scenario as we would like to keep our dashboard updated with the most recent data.

If a date is provided as the second argument then it is assigned to `last_date`. This is useful when you want to test a model on historical data. By default, the `all_data.csv` and `summary.csv` files are overwritten. If you attempt to execute `update.py` with any further arguments an error will be raised with a message returned on how to execute it properly.

The rest of the code block runs all of our previous work in just a few lines of code. It instantiates and calls the `run` method from `PrepareData`, `CasesModel`, and `DeathsModel`, before calling the `combine_all` and `create_summary_table` to create the final data files.

```python
# update.py file
import sys
from prepare import PrepareData, combine_all_data, create_summary_table
from models import CasesModel, DeathsModel, general_logistic_shift

# Parameters for CasesModel - Feel free to change these
N_TRAIN = 60   # Number of observations used in training
N_SMOOTH = 15  # Number of observations used in smoothing
N_PRED = 56    # Number of new observations to predict
L_N_MIN = 5    # Number of days of exponential growth for L min boundary
L_N_MAX = 50   # Number of days of exponential growth for L max boundary

# Parameters for DeathsModel - Feel free to change these
LAG = 15       # Number of days to lag cases for calculation of CFR
PERIOD = 30    # Number of days to total for CFR

if __name__ == "__main__":
    if len(sys.argv) == 1:
        last_date = None
    elif len(sys.argv) == 2:
        last_date = sys.argv[1]
    else:
        raise TypeError(
            """
            When calling `python update.py` from the command line,
            pass 0 or 1 arguments.
                0 arguments: make prediction for latest data (downloads latest data)
                1 argument: provide the last date that the model will see, i.e. 20200720
            """
        )
    data = PrepareData().run()
    cm = CasesModel(
        model=general_logistic_shift,
        data=data,
        last_date=last_date,
        n_train=N_TRAIN,
        n_smooth=N_SMOOTH,
        n_pred=N_PRED,
        L_n_min=L_N_MIN,
        L_n_max=L_N_MAX,
    )
    cm.run()

    dm = DeathsModel(data=data, last_date=last_date, cm=cm, lag=15, period=30)
    dm.run()

    df_all = combine_all_data(cm, dm)
    create_summary_table(df_all, cm.last_date)
```

### Run all of the code now

Go to your command line and run `python update.py` twice, once within each of the `project` and `notebooks` directory to update the data in each one and to verify that it works.